# Bayesian networks

A *bayesian network* is a directed acyclic graph such that each node represents a random variable and has an associated probability distribution: 

* for a node without any predecessors (i.e., without any incoming edges), it is an unconditional probability distribution
* for a node with predecessors (i.e., with some incoming edges), it is a conditional probability distribution, conditioning on the variables represented by the other end of the incoming edges. We will call these variables *parents*.

The picture below shows a bayesian networks with 5 variables: two unconditional (*Bulglary*, *Earthquake*), and three conditional:

* *Alarm*, conditioning on *Bulglary* and *Earthquake* (i.e., *Bulgary* and *Earthquake* are parents for *Alarm*),
* *JohnCalls*, conditioning on *Alarm*
* *MaryCalls*, also conditioning on *Alarm*

In this particular example, each variables in a boolean variable, that is it can be either *true* or *false*. The probability distributions are incomplete, only specifying the probability for a variable being *true*. Fortunately, it is sufficient to subtract the given values from, to arrive at probabilities for *false* and thus complete the distributions.

For the conditional probability distributions, each row represents a separate distribution conditioning on the values of the parent variables given in the left-hand side of the distribution. For example:
$$ P(Alarm = true | Bulglary = true, Earhquake = false) = 0.94 $$
and by following the previous paragraph:
$$ P(Alarm = false | Bulglary = true, Earhquake = false) = 1 - P(Alarm = true | Bulglary = true, Earhquake = false) = 1 - 0.94 = 0.06 $$

![](aima-images/aima4_fig13_2.png)

Let us now try to come up with a data structure suitable for representing a bayesian network. We will use strings as variable names and allow more-or-less any values to be the elements of the domain of the variable. To make the code somewhat more readable, we will use Python's `typing` module and introduce some type aliases:

In [79]:
from typing import Tuple, Iterable, Hashable

Variable = str
DomainElement = Hashable # anything that can be a key of a dictionary

Now, what is a probability distribution? A mapping from the elements of a domain to probabilities. In Python's terms: a dictionary `dict` from `DomainElement` to `float`

In [80]:
ProbabilityDistribution = dict[DomainElement, float]

Conditional probability distributions is somewhat more complicated, as it requires a separate probability distribution `ProbabilityDistribution` for each assignment of values to conditions. We will represent each assignment `Assignment` as a tuple of `DomainElements` of arbitrary lenth, and the conditional probability distribution `ConditionalProbabilityDistribution` as a dictionary `dict` from `Assignment`s to `ProbabilityDistribution`s. In order to ensure that we know which position in `Assignment` represents which variable, we will introduce an auxiliary type `Parents`, being a tuple of `Variable`s of arbitrary length. It serves the same purpose as the header in the top-left corner in the conditional probability distributions given in the picture above.

In [81]:
Assignment = Tuple[DomainElement, ...]
ConditionalProbabilityDistribution = dict[Assignment, ProbabilityDistribution]
Parents = Tuple[Variable, ...]

The following class `BayesianNet` uses these type aliases to define a class representing a bayesian network along with some auxiliary functions. The detailed description is given below it, along with some examples how to use it.

In [82]:
class BayesianNet:        
    _unconditional: dict['Variable', 'ProbabilityDistribution']
    _conditional: dict['Variable', tuple['Parents', 'ConditionalProbabilityDistribution']]
    _domain: dict['Variable', Iterable['DomainElement']]
    
    def __init__(self):
        self._unconditional = {}
        self._conditional = {}
        self._domain = {}        
        
    def _check_and_normalize(self, distribution: 'ProbabilityDistribution') -> 'ProbabilityDistribution':
        assert len(distribution.keys()) >= 2
        if None in distribution.values():
            rem = sum([f for f in distribution.values() if f is not None])
            assert 0 <= rem < 1
            noneKey = [k for k, v in distribution.items() if v is None]
            assert len(noneKey) == 1
            distribution[noneKey[0]] = 1 - rem
        assert all(0<v<1 for v in distribution.values())
        assert sum(distribution.values()) == 1
        return distribution
        
    def addUnconditionalVariable(self, name: 'Variable', distribution: 'ProbabilityDistribution') -> None:        
        assert name not in self._conditional
        assert name not in self._unconditional
        distribution = self._check_and_normalize(distribution)
        self._unconditional[name] = distribution
        self._domain[name] = set(distribution.keys())
    
    def addConditionalVariable(self, name: 'Variable', parents: 'Parents', cpt: 'ConditionalProbabilityDistribution') -> None:        
        assert name not in self._conditional
        assert name not in self._unconditional
        assert isinstance(parents, tuple)
        assert len(parents) > 0
        assert all(len(parents) == len(k) for k in cpt.keys())
        domain = set(next(iter(cpt.values())).keys())
        assert all(v.keys() == domain for v in cpt.values())        
        cpt = {k: self._check_and_normalize(distribution) for k, distribution in cpt.items()}
        self._conditional[name] = (parents, cpt)
        self._domain[name] = domain
    
    def addBooleanUnconditionalVariable(self, name: 'Variable', pTrue: float) -> None:
        assert 0 < pTrue < 1
        self.addUnconditionalVariable(name, {True: pTrue, False: 1-pTrue})
        
    def addBooleanConditionalVariable(self, name: 'Variable', parents: 'Parents', cpt: 'dict[Assignment, float]'):
        cpt = {k: {True: v, False: 1-v} for k, v in cpt.items()}
        self.addConditionalVariable(name, parents, cpt)
        
    def domain(self, name: 'Variable') -> 'Iterable[DomainElement]':
        return self._domain[name]
    
    def variables(self) -> Iterable['Variable']:
        return self._conditional.keys() | self._unconditional.keys()
    
    def parents(self, name: 'Variable') -> 'Parents':
        if name in self._conditional:
            return self._conditional[name][0]
        else:
            return []
    
    def p(self, name: 'Variable', value: 'DomainElement', condition: dict['Variable', 'DomainElement']):
        if name in self._conditional:
            parents, cpt = self._conditional[name]
            assert all(p in condition for p in parents)
            condition = tuple(condition[p] for p in parents)
            dist = cpt[condition]
        else:
            assert name in self._unconditional
            dist = self._unconditional[name]
        return dist[value]

We will create a network corresponding to the one given in the picture above. For easier reading the same picture is repeated below.

![](aima-images/aima4_fig13_2.png)

We begin by creating a new network in the variable `bn` and adding an unconditional variable *B* using `addUncoditionalVariable`. Its first argument is the variable, the second is a probability distribution. As this variable is a boolean variable, it is a dictionary from the boolean values to probabilities. It is allowed for one of the probabilities to remain unspecified (using `None`), in which case it is automatically computed from the remaining values.

In [83]:
bn = BayesianNet()
bn.addUnconditionalVariable('B', {True: 0.001, False: None})

Observe that we require the random variable to have at least two elements of the domain, and the numeric values given must be probabilities and not arbitrary numbers. The following cell contains commented-out code, you can uncomment each line separately, execute the cell and observe that an exception was raised.

In [84]:
#bn.addUnconditionalVariable('B1', {True: 1})
#bn.addUnconditionalVariable('B1', {True: -1, False: 2})
#bn.addUnconditionalVariable('B1', {True: None, False: None})

For boolean variables it is somewhat cumbersome to write all this boilerplate by hand, so there's an auxiliary method `addBooleanUnconditionalVariable`, which expects only a variable name and the probability for *true*. The following line adds an unconditional variable `E` with $P(E=true) = 0.002$ and $P(E=false) = 0.998$ (computed automatically)

In [85]:
bn.addBooleanUnconditionalVariable('E', 0.002) # the same as bn.addUnconditionalVariable('E', {True: 0.002, False: None})

Now let us add the conditional variable *A*, following the distribution given in the picture. The method `addConditionalVariable` expects three parameters: a variable name, a tuple of variable names representing parents of the variable and a conditional probability distribution, i.e., a dictionary from tuples of domain elements to probability distributions. The keys in the dictionary `cpd` are each a pair of boolean values, and their meaning is given by the variable `parents`: the first value is the value for the variable *B* and the other for varialbe *E*. The probability distributions follow the same conventions as described earlier for `addUnconditionalVariable`.

In [86]:
parents = ('B', 'E')
cpd = {(True, True): {True: .95, False: None}, 
       (True, False): {True: .94, False: None}, 
       (False, True): {True: .29, False: None}, 
       (False, False): {True: .001, False: None}}

bn.addConditionalVariable('A', parents, cpd)

Again, writing the boilerplate for boolean variables is cumbersome. Enters `addBooleanConditionalVariable`, which expects only probabilities for true in the dictionary, instead of complete probability distributions. The following two lines add two variables, *J* and *M*, both having a single parent: *A*. Observe that the second argument is still a tuple (consisting of a single element) and similarly the keys in the conditional probability distribution are tuples of a single element.

In [87]:
bn.addBooleanConditionalVariable('J', ('A',), {(True,): .9, (False,): .05})
bn.addBooleanConditionalVariable('M', ('A',), {(True,): .7, (False,): .01})

Let us see what variables are available in the networks:

In [88]:
bn.variables()

{'A', 'B', 'E', 'J', 'M'}

What is the domain of the variable *A*?

In [89]:
bn.domain('A')

{False, True}

What are the parents of the variable *A*?

In [90]:
bn.parents('A')

('B', 'E')

What is the probability $P(A=true | B=true, E=false)$?

In [91]:
bn.p('A', True, {'B': True, 'E': False})

0.94

Observe that the method `p` expects three arguments: the variable, the value we are interested in, and the conditions. There may be more information than necessary in the third argument, but there cannot be less. For example, the following cell works just fine, returning the probability of the alarm ringing given that there was a bulgary, there was no earthquake and both Mary and John called, the last two pieces of information being superfluous.

In [92]:
bn.p('A', True, {'B': True, 'E': False, 'M': True, 'J': True})

0.94

`p` will not, however, be able to deal with not enough information. The following cell will fail, as it is unknown whether there was a bulgary or not.

In [93]:
bn.p('A', True, {'E': False})

`p` works fine for unconditional variables and simply ignores the last argument (both cells below should return the same value):

In [94]:
bn.p('B', True, {'E': False})

0.001

In [95]:
bn.p('B', True, None)

0.001

## Task 1: Topological sorting

A *topological order* is any order such that for any variable, all its parents are earlier in the order. It is a stepping stone for an inference algorithm further down the line. The following function is checks whether the given ordering is, in fact, a topological order. If it is not, it raises an exception.

In [96]:
def verify_topological_order(bn: BayesianNet, ordering: list['Variable']):
    for i, var in enumerate(ordering):
        assert all(p in ordering[:i] for p in bn.parents(var))

For example, *B, E, A, J, M* is a topological ordering for the network we constructed earlier, while *A, B, E, J, M* is not (so the second cell should fail)

In [97]:
verify_topological_order(bn, ['B', 'E', 'A', 'J', 'M'])

In [98]:
verify_topological_order(bn, ['A', 'B', 'E', 'J', 'M'])

Complete the following function `topological_sort` such that it returns a list of variables of the given `BayesianNet` in a *topological order*. Topological sorting is quite similar to forward chaining with Horn clauses, but without the risk of cycles, so you should have no problem coming up with a solution. (Additional reading: page 434 in AIMA 4ed, in Chapter 13.2)

In [99]:
def topological_sort(bn: BayesianNet) -> list['Variable']:
    result = []
    visited = set()
    vars_list = list(bn.variables())

    while len(result) < len(vars_list):
        for var in vars_list:
            if var not in visited:
                if all(p in visited for p in bn.parents(var)):
                    visited.add(var)
                    result.append(var)
                    break
    return result

Lets see if your code works correctly! The following cell should not fail.

In [100]:
verify_topological_order(bn, topological_sort(bn))

In [101]:
topological_sort(bn)

['E', 'B', 'A', 'M', 'J']

## Task 2: Implement inference algorithm for Bayesian Networks

Complete the following function `enumeration_ask` by implementing the algorithm *ENUMERATION-ASK* given in Figure 13.11 of AIMA 4ed. It should return a probability distribution for the variable `X` given the assignment (results of observation) `e` and the Bayesian network `bn`.

In [102]:
def enumeration_ask(X: 'Variable', e: dict['Variable', 'DomainElement'], bn: BayesianNet) -> 'ProbabilityDistribution':
    qx = {}
    for xi in bn.domain(X):
        e_extended = e.copy()
        e_extended[X] = xi
        vars_list = topological_sort(bn)
        qx[xi] = enumerate_all(vars_list, e_extended, bn)

    total = sum(qx.values())
    return {k: v / total for k, v in qx.items()}

def enumerate_all(vars_list, e, bn):
    if not vars_list:
        return 1.0

    Y = vars_list[0]
    rest = vars_list[1:]

    if Y in e:
        return bn.p(Y, e[Y], e) * enumerate_all(rest, e, bn)
    else:
        total = 0
        for yi in bn.domain(Y):
            e_extended = e.copy()
            e_extended[Y] = yi
            total += bn.p(Y, yi, e_extended) * enumerate_all(rest, e_extended, bn)
        return total

Assume that both John and Mary called. AIMA gives the probability for an ongoing bulgary of around 0.284 and 0.716 for the lack of bulgary (see page 446 of AIMA 4ed). Lets see if you agree (the following cell should not fail)

In [103]:
prob = enumeration_ask("B", {"J": True, "M": True}, bn)
assert abs(prob[True] - 0.284) <= 0.001
assert abs(prob[False] - 0.716) <= 0.001

## Task 3: Biased coins

Complete the following function `coins` such that it returns a Bayesian networks representing the following problem: We have a bag of three biased coins *A*, *B*, and *C* with probabilities of coming up heads of `pa`, `pb`, and `pc`, respectively. One coin is drawn randomly from the bag (with equal likelihood of drawing each of the three coins), and then the coin is flipped three times to generate the outcomes *X1*, *X2*, and *X3*. Create an unconditional variable `Coin` and define it over the domain of `'A'`, `'B'`, `'C'`. Create conditional variables `X1`, `X2`, `X3`, and define them over a binary domain consisting of the values `'H'` and `'T'`.


In [104]:
def coins(pa: float, pb: float, pc: float) -> BayesianNet:
    bn = BayesianNet()
    bn.addUnconditionalVariable('Coin', {'A': 1/3, 'B': 1/3, 'C': 1/3})

    coin_probs = {'A': pa, 'B': pb, 'C': pc}

    for x in ['X1', 'X2', 'X3']:
        cpt = {
            ('A',): {'H': coin_probs['A'], 'T': 1 - coin_probs['A']},
            ('B',): {'H': coin_probs['B'], 'T': 1 - coin_probs['B']},
            ('C',): {'H': coin_probs['C'], 'T': 1 - coin_probs['C']}
        }
        bn.addConditionalVariable(x, ('Coin',), cpt)

    return bn

The following cell computes the probability distribution for the variable `Coins` with probabilities of coming up heads, respectively, 30%, 60% and 75% assuming that the coin drawn from the bag yielded heads, heads and tails.

In [105]:
coins_bn = coins(.3, .6, .75)
pd = enumeration_ask('Coin', {'X1': 'H', 'X2': 'H', 'X3': 'T'}, coins_bn)
pd

{'B': 0.4142394822006472, 'A': 0.18122977346278316, 'C': 0.4045307443365696}

Let's see if your solution is correct:

In [106]:
assert abs(pd['A'] - 0.181) <= 0.005
assert abs(pd['B'] - 0.414) <= 0.005
assert abs(pd['C'] - 0.405) <= 0.005
assert pd['B'] > pd['C'] > pd['A']

---
The pictures are from "Artificial Intelligence: A Modern Approach" 4rd ed. Task 3 is adapted from chapter 14 of "Artificial Intelligence: A Modern Approach" 3rd ed.

The typing in this notepad is a bit sloppy. There should be a separate type `IncompleteProbabilityDistribution = dict[DomainElement, Optional[float]]` explicitly allowing for `None` instead of a probability value. It would, however, make the code harder to follow.